In [4]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle 


In [5]:
##load the dataset 
df = pd.read_csv("Churn_Modelling.csv")

In [6]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data prprocessing 

In [7]:
df = df.drop(["RowNumber","CustomerId","Surname"],axis=1)


In [8]:
label_encoder_gender= LabelEncoder()
df["Gender"]= label_encoder_gender.fit_transform(df["Gender"])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder()
geography_OHE= OHE.fit_transform(df[["Geography"]]).toarray()
geography_OHE

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
OHE.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geography_OHE,columns=OHE.get_feature_names_out())

In [12]:
df = pd.concat([df,geo_encoded_df],axis=1)

In [13]:
df= df.drop("Geography",axis=1)

In [32]:
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)
with open("OHE.pkl","wb") as file:
    pickle.dump(OHE,file )


In [15]:
## train test split 
X = df.drop("Exited",axis =1)
y = df["Exited"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
## save pickle file 
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN implementation 

In [17]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime 

In [18]:
(X_train.shape[1],)

(12,)

In [19]:
### build our ANN model 
model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)), ##Hl1 connected with input layer 
    Dense(32,activation= "relu"),
    Dense(1,activation= "sigmoid") ## output layer 
])

d:\python\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
opt= tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [22]:
### compile the model 
model.compile(optimizer=opt ,loss= "binary_crossentropy",metrics= ["accuracy"])

In [23]:
## set up the Tensorboard 
log_dir = "logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback= TensorBoard(log_dir= log_dir,histogram_freq=1)

In [24]:
## set up eartly stopping 
early_stopping_callback = EarlyStopping(monitor="val_loss",patience= 10 ,restore_best_weights=True)

In [25]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

print("Sample y_train:", y_train[:10])
print("Sample y_test:", y_test[:10])


X_train shape: (8000, 12)
y_train shape: (8000,)
X_test shape: (2000, 12)
y_test shape: (2000,)
Sample y_train: 9254    0
1561    0
1670    1
6087    1
6669    1
5933    1
8829    0
7945    0
3508    0
2002    0
Name: Exited, dtype: int64
Sample y_test: 6252    0
4684    0
1731    0
4742    0
4521    0
6340    0
576     0
5202    1
6363    0
439     0
Name: Exited, dtype: int64


In [26]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8357 - loss: 0.3960 - val_accuracy: 0.8540 - val_loss: 0.3541
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8545 - loss: 0.3567 - val_accuracy: 0.8520 - val_loss: 0.3454
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8594 - loss: 0.3456 - val_accuracy: 0.8615 - val_loss: 0.3414
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8591 - loss: 0.3444 - val_accuracy: 0.8560 - val_loss: 0.3452
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8612 - loss: 0.3383 - val_accuracy: 0.8625 - val_loss: 0.3468
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8620 - loss: 0.3358 - val_accuracy: 0.8580 - val_loss: 0.3398
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8640 - loss: 0.3333 - val_accuracy: 0.8535 - val_loss: 0.3541
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8649 - loss: 0.3326 - val_accu

In [27]:
model.save("model.h5")

In [28]:
## load Tensorboard Extension 
%load_ext tensorboard

In [29]:
%tensorboard --logdir logs/fit20250815-024053

Reusing TensorBoard on port 6006 (pid 15240), started 0:37:37 ago. (Use '!kill 15240' to kill it.)

In [30]:
## load the pickle 
model = load_model("model.h5")

## load the encoder and scaler 
with open("label_encoder_gender.pkl","rb")as file:
    label_encoder_gender = pickle.load(file)
with open("geography_OHE.pkl","rb")as file:
    geography_OHE = pickle.load(file)
with open("scaler.pkl","rb")as file:
    scaler  = pickle.load(file)
 

NameError: name 'load_model' is not defined